In [42]:
from Bio import Entrez
from Bio import SeqIO
import re

keyword="(Vibrio phage|vibriophage).*genome"
accs=[]
titles=[]
acdict={}

splist=open("/Users/jmb/Downloads/nuccore_result-1.txt").readlines()
for i in range(1,len(splist),4):
    if re.search(keyword, splist[i]):
        name=splist[i].split(",")[0].split(".")[1]
        name2=re.sub(r"complete genome\n|genomic sequence\n","", name)
        if name not in titles:
            titles.append(name2)
            acc=splist[i+2].split(" ")[0]
            accs.append(acc)
            acdict[acc]=name2
accs_str=",".join(accs)

import os

#os.mkdir("./VibGenomes")
Entrez.email="julia.brown@einstein.yu.edu"
for a in accs:
    handle=Entrez.efetch(db="nucleotide", id=str(a), rettype="gb", retmode="text")
    filename="./VibGenomes/"+a+".gbk"
    out=open(filename, "w")
    out.write(handle.read())
    out.close()
    handle.close()

for a in accs:
    SeqIO.convert("./VibGenomes/"+a+".gbk","gb","./VibGenomes/"+a+".fasta","fasta")
    print(a+" converted")

NC_019713.1 converted
KJ572844.2 converted
KP280062.1 converted
NC_019722.1 converted
KM612265.1 converted
KM612263.1 converted
KM612262.1 converted
KM612260.1 converted
NC_027393.1 converted
KM612261.1 converted
NC_015159.1 converted
NC_027397.1 converted
NC_013651.1 converted
KP280063.1 converted
NC_015157.1 converted
NC_015158.1 converted
NC_016567.1 converted
NC_005891.1 converted
KP671755.1 converted
KJ545483.2 converted
HQ317393.2 converted
NC_005083.2 converted
NC_021073.1 converted
NC_020488.1 converted
JQ340389.1 converted
NC_005879.1 converted
NC_003907.2 converted
NC_024369.2 converted
NC_025822.1 converted
KF800937.1 converted
NC_022747.1 converted
NC_019518.1 converted
NC_019457.1 converted
KT160311.1 converted
NC_027118.1 converted
NC_026610.1 converted
NC_003313.1 converted
NC_023569.1 converted
KJ936628.1 converted
NC_023863.2 converted
KJ502657.1 converted
NC_023605.1 converted
NC_023568.1 converted
NC_010275.1 converted
NC_004456.1 converted
JF713456.1 converted
HQ641

In [44]:
from Bio.SeqUtils import GC

taxdict={}
sizedict={}
out=open("vib.genomeinfo.txt","w")

for phage in accs:
    gbk=SeqIO.parse("./VibGenomes/"+phage+".gbk", "genbank")
    for rec in gbk:
        taxonomy=rec.annotations["taxonomy"]
        if len(taxonomy)>4:
            tax=taxonomy[3]
        else:
            tax=taxonomy[-1]
        taxdict[phage]=tax
        sizedict[phage]=len(rec.seq)
        gc_content=GC(rec.seq)
        out.write(phage+"\t"+acdict[phage]+"\t"+tax+"\t"+str(len(rec.seq))+"\t"+str(gc_content)+"\n")
out.close()

In [45]:
#os.mkdir("./VibGenomes/vib_tRNA/")
out=open("vib_trna_scan.sh","w")
for phage in accs:
    out.write("tRNAscan-SE -o ./VibGenomes/vib_tRNA/"+phage+"trnas.txt -G -D ./VibGenomes/"+phage+".fasta\n")
out.close()
!bash vib_trna_scan.sh


tRNAscan-SE v.1.3.1 (January 2012) - scan sequences for transfer RNAs

  Please cite: 
	Lowe, T.M. & Eddy, S.R. (1997) "tRNAscan-SE: A program for
	improved detection of transfer RNA genes in genomic sequence"
	Nucl. Acids Res. 25: 955-964.

  This program uses a modified, optimized version of tRNAscan v1.3
  (Fichant & Burks, J. Mol. Biol. 1991, 220: 659-671),
  a new implementation of a multistep weight matrix algorithm
  for identification of eukaryotic tRNA promoter regions
  (Pavesi et al., Nucl. Acids Res. 1994, 22: 1247-1256),
  as well as the RNA covariance analysis package Cove v.2.4.2
  (Eddy & Durbin, Nucl. Acids Res. 1994, 22: 2079-2088).

------------------------------------------------------------
Sequence file(s) to search:  ./VibGenomes/NC_019713.1.fasta
Search Mode:                 General
Results written to:          ./VibGenomes/vib_tRNA/NC_019713.1trnas.txt
Output format:               Tabular
Searching with:              tRNAscan + EufindtRNA -> Cove
Covariance mo

In [46]:
import glob

names=glob.glob("./VibGenomes/vib_tRNA/*")

tRNA_phages=[]

for n in accs:
    tfile="./VibGenomes/vib_tRNA/"+n+"trnas.txt"
    if os.path.getsize(tfile)>0:
        #print("phage "+phage+" has tRNAs")
        tRNA_phages.append(n)
    #else:
     #   print("phage "+phage+" does not have tRNAs")

print("there are a total of "+str(len(names))+" phages but only "+str(len(tRNA_phages))+" have tRNAs")

out=open("vib_tRNA_summary.txt","w")
out.write("phage_short\tphage_full\tbound1\tbound2\ttRNA\tanti_codon\tintron_bound1\titron_bound2\tscore\n")

for phage in tRNA_phages:
    info=open("./VibGenomes/vib_tRNA/"+phage+"trnas.txt").readlines()
    for line in info[3:]:
        vec=line.split("\t")
        
        out.write(acdict[phage]+"\t"+vec[0]+"\t"+vec[2]+"\t"+vec[3]+"\t"+vec[4]+"\t"+vec[5]+"\t"+vec[6]+"\t"+vec[7]+"\t"+vec[8])
out.close()

there are a total of 92 phages but only 11 have tRNAs


In [14]:
%load_ext rpy2.ipython
%R library(dplyr);library(magrittr); library(ggplot2)


Attaching package: ‘dplyr’

The following object is masked from ‘package:stats’:

    filter

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



<StrVector - Python:0x10afec8c0 / R:0x108307698>
[str, str, str, ..., str, str, str]

In [34]:
for phage in accs:
    gbk=SeqIO.parse("./VibGenomes/"+phage+".gbk", "genbank")
    print(rec)

ID: KC131130.1
Name: KC131130
Description: Vibrio phage VP4B, partial genome.
Number of features: 211
/comment=##Assembly-Data-START##
Assembly Method       :: SOAPdenovo v. 1.06
Sequencing Technology :: Illumina
##Assembly-Data-END##
/sequence_version=1
/source=Vibrio phage VP4B
/taxonomy=['Viruses', 'dsDNA viruses, no RNA stage', 'Caudovirales', 'Myoviridae']
/keywords=['']
/references=[Reference(title='Complete genome sequence of a novel phiKZ-like Vibrio phage', ...), Reference(title='Direct Submission', ...)]
/accessions=['KC131130']
/data_file_division=PHG
/date=26-DEC-2013
/organism=Vibrio phage VP4B
/gi=432142535
Seq('GTCAACCAAAAAGACATCCTAAGTGGACACTCTTCGAAATGCACCAACGAATTC...ACG', IUPACAmbiguousDNA())
ID: KC131130.1
Name: KC131130
Description: Vibrio phage VP4B, partial genome.
Number of features: 211
/comment=##Assembly-Data-START##
Assembly Method       :: SOAPdenovo v. 1.06
Sequencing Technology :: Illumina
##Assembly-Data-END##
/sequence_version=1
/source=Vibrio phage VP4B
/